In [1]:
import logging
import asyncio
import pandas as pd
from IPython.display import display, Markdown

from datetime import datetime

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials

from tastytrade.messaging.processors import TelegrafHTTPEventProcessor, RedisEventProcessor
from tastytrade.config import RedisConfigManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-08-09 16:13:07 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20250809.log


# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())
    handler.add_processor(RedisEventProcessor())

2025-08-09 16:13:09 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-08-09 16:13:09 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-08-09 16:13:09 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 77.020ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.244.143', 443))]


2025-08-09 16:13:09 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x73117c360e50> starts SSL handshake
2025-08-09 16:13:09 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x73117c360e50>: SSL handshake took 42.4 ms
2025-08-09 16:13:09 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=78, family=2, type=1, proto=6, laddr=('172.18.0.7', 56900), raddr=('170.76.244.143', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0x731122fcb1d0>, <aiohttp.client_proto.ResponseHandler object at 0x731122fcd4e0>)
2025-08-09 16:13:09 - INFO:tastytrade.connections.requests:148:Session created successfully
2025-08-09 16:13:09 - DEBUG:asyncio:848:Get address info tasty-openapi-ws.dxfeed.com:443, type=<SocketKind.SOCK_STREAM: 1>
2025-08-09 16:13:09 - INFO:asyncio:856:Getting address info tasty-openapi-ws.dxfeed.com:443, type=<SocketKind.SOCK_STREAM: 1> took 129.737ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('13.226.94.47', 44

2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:9
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:11
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:99
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:5:COMPACT
2025-08-09 16:13:09 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:7:COMPACT
2025-08-09 16:13:09 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [3]:
start_time = datetime(2025, 8, 8)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
# symbols = ["BTC/USD:CXTALP", "NVDA",]
# symbols = ["AAPL", "QQQ"]  # 
# symbols = ["SPY", "SPX"]  # 
intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=h}
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=30m}
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=15m}
2025-08-09 16:13:10 - INFO:tastytrade.connections.sockets:236:A

In [4]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

2025-08-09 16:13:15 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:15 - INFO:root:124:Forward-fill added 4 events for BTC/USD:CXTALP{=15m}
2025-08-09 16:13:15 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:15 - INFO:root:124:Forward-fill added 28 events for BTC/USD:CXTALP{=5m}
2025-08-09 16:13:15 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:15 - INFO:root:124:Forward-fill added 207 events for BTC/USD:CXTALP{=m}
2025-08-09 16:13:15 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:15 - INFO:root:124:Forward-fill added 10 events for NVDA{=m}
2025-08-09 16:13:16 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:16 - INFO:root:124:Forward-fill added 43 events for AAPL{=m}
2025-08-09 16:13:16 - INFO:root:120:Successfully flushed and closed InfluxDB write API
2025-08-09 16:13:16 - INFO:root:124:Forward-fill added 

# Check | Market Data feeds

In [5]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
display(
    dxlink.router.handler[Channels.Candle]
    .processors["feed"]
    .frames[f"{symbol}"]
    .tail(5)
    .sort(by="time", descending=True)
)

**Candle Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC/USD:CXTALP{=5m}""",2025-08-09 16:10:00,0,7536618773466316800,0,241,116889.93,116889.93,116842.82,116851.12,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 16:05:00,0,7536617484976128000,0,563,116899.56,116945.06,116889.93,116889.93,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 16:00:00,0,7536616196485939200,0,740,116921.96,116977.96,116887.73,116901.49,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 15:55:00,0,7536614907995750400,0,361,116863.2,116920.84,116861.0,116920.84,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-08-09 15:50:00,0,7536613619505561600,0,474,116869.23,116892.86,116838.61,116863.79,null,null,null,0.0,null,null


In [6]:
display(Markdown(f"**Quote Feed:** {symbol}"))

display(dxlink.router.handler[Channels.Quote].processors["feed"].pl)

**Quote Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""SPY""",610.77,638.66,null,null
"""QQQ""",565.0,575.99,null,null
"""SPX""",6338.53,6434.76,null,null
"""NVDA""",182.81,183.0,null,null
"""BTC/USD:CXTALP""",116259.78,117441.1,0.05,0.05
"""AAPL""",229.85,230.0,null,null


In [7]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [8]:
await dxlink.close()

2025-08-08 17:57:00 - INFO:tastytrade.connections.sockets:450:Listener task cancelled
2025-08-08 17:57:00 - INFO:tastytrade.connections.sockets:450:Keepalive task cancelled
2025-08-08 17:57:00 - INFO:tastytrade.connections.routing:58:Initiating cleanup...
2025-08-08 17:57:00 - INFO:tastytrade.connections.routing:82:Cleanup completed
2025-08-08 17:57:00 - INFO:tastytrade.connections.requests:175:Session closed
2025-08-08 17:57:00 - INFO:tastytrade.connections.sockets:444:Connection closed and cleaned up
